In [566]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
import functions
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

In [567]:
#load pcd file, filter, downsample
#pcdn = o3d.io.read_point_cloud("loop.pcd")

voxel_size = 0.4
pcd = o3d.io.read_point_cloud("final_cropped_ground_align.pcd")
pcd.estimate_normals()
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcd.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
#o3d.visualization.draw_geometries([pcd])

In [568]:
pcd_flat = functions.filter_by_normal(pcd)

#o3d.visualization.draw_geometries([pcd_flat])

In [569]:
oboxes = pcd_flat.detect_planar_patches(
normal_variance_threshold_deg=50,
coplanarity_deg=85,
outlier_ratio=0.75,
min_plane_edge_length=2,
min_num_points=10,
search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50))

print("Detected {} patches".format(len(oboxes)))

geometries = []
meshes = []
for obox in oboxes:

    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(obox.color)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
    normals = np.asarray(mesh.vertex_normals)

    
    meshes.append(mesh)
    geometries.append(obox)



Detected 24 patches


In [570]:
hor_patches, ver_patches = devide_meshes_hor_ver(meshes)
o3d.visualization.draw_geometries(ver_patches)
o3d.visualization.draw_geometries(hor_patches)
print(len(hor_patches), len(ver_patches))

6 18


In [571]:


corr, corr_tuples = group_corresponding_meshes([],[],hor_patches, "horizontal")

corr, corr_tuples = group_corresponding_meshes(corr, corr_tuples, ver_patches, "vertical")


In [572]:




midpoints, marker_meshes = find_midpoints(corr_tuples)

print(midpoints)
o3d.visualization.draw_geometries(corr + marker_meshes)

[array([-7.07151553e+00,  5.48340336e+00,  3.84389387e-03]), array([-7.43322789e+00,  4.57257718e+00,  5.30192719e-03]), array([-4.26263094e+01,  6.91031496e-01,  5.47953889e-03]), array([-1.84313939, -0.19252207,  0.00512816]), array([-2.29123088e+01,  2.66933538e-01,  5.32966019e-03]), array([-1.09801688e+01, -2.62486501e-03,  5.30903716e-03]), array([-3.92460838e+01,  3.79474002e-01,  5.04139822e-03]), array([-3.39843344e+01, -2.81458113e+00,  7.14665834e-03]), array([-8.08591727e+00,  5.65995488e+00,  5.51892960e-03]), array([-3.51129990e+01, -3.07914625e+00,  7.27719865e-03])]


In [573]:
#bb = pcd_flat.get_axis_aligned_bounding_box()
#bb.color = np.asarray([0,0,1])
#uniform_pc = create_uniform_pc_from_bb(bb, 0.2, [1,0,0])
hull, _ = pcd_flat.compute_convex_hull()

uniform_pc = hull_to_uniform_pc(hull, 0.2, [1,0,0])

o3d.visualization.draw_geometries([pcd_flat, uniform_pc, bb])

pcd_flat = create_uniform_pc(pcd_flat, 0.1, [0,0,0])
o3d.visualization.draw_geometries([pcd_flat])

In [577]:
# search multiple iterations until boundary found
search_radius = np.ceil(np.sqrt((voxel_size*2)**2+(voxel_size*2)**2)*10000)/10000

uniform_pc.paint_uniform_color([1,0,0])
pcd_flat.paint_uniform_color([0,0,0])

PointCloud with 4068 points.

In [578]:


void_area, known_points = extract_void_area(uniform_pc, pcd_flat, midpoints)
o3d.visualization.draw_geometries([void_area, pcd_flat]+marker_meshes)

In [579]:
valid_area = uniform_pc.select_by_index(list(known_points))
valid_area_voxel = o3d.geometry.VoxelGrid.create_from_point_cloud(valid_area, 0.2)
o3d.visualization.draw_geometries([valid_area_voxel, pcd_flat])